In [1]:
!pip install accelerate>=0.20.1

# 필요한 모듈 install

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
 !pip install transformers
 !pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 58.8 MB/s eta 0:00:00


In [4]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    BartConfig
)
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# 데이터 로드 - 데이터 개수 110개 - train:100, val:10

In [81]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/data/news_summary_train_dataset.tsv', sep='\t')
df = train_df[:110]

# 모델, tokenizer 설정

In [6]:
model_name = 'digit82/kobart-summarization'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# 데이터 전처리
- target 데이터에 bos, eos 붙이기
- max_length : 1024

In [82]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, num_data=None, max_length=1024):
        self.num_data = num_data
        self.max_length = max_length
        self.data = data[:num_data]
        self.tokenizer = tokenizer
        self.input_tokenized, self.target_tokenized = self.tokenizing()

    def tokenizing(self):
        input_tokenized = []
        target_tokenized = []

        for i in range(len(self.data)):
            input_text = self.data.iloc[i, 0]
            target_text = self.data.iloc[i, 1]

            input_ids = self.tokenizer(input_text, padding="max_length", max_length=self.max_length, truncation=True)["input_ids"]
            target_ids = self.tokenizer('<s>' + target_text + '</s>', padding="max_length", max_length=self.max_length, truncation=True)["input_ids"]

            input_tokenized.append(input_ids)
            target_tokenized.append(target_ids)

        return torch.LongTensor(input_tokenized), torch.LongTensor(target_tokenized)

    def __len__(self):
        return len(self.input_tokenized)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_tokenized[idx],
            'labels': self.target_tokenized[idx]
        }

In [83]:
data_train = df.iloc[:100, :]
data_val = df.iloc[100:110, :]

dataset_train = CustomDataset(data_train,tokenizer=tokenizer)
dataset_val = CustomDataset(data_val,tokenizer=tokenizer)

In [72]:
dataset_train[0]

{'input_ids': tensor([15634, 11764, 16186,  ...,     3,     3,     3]),
 'labels': tensor([    0, 15529,   271,  ...,     3,     3,     3])}

In [73]:
dataset_val[0]

{'input_ids': tensor([14437, 15639, 16415,  ...,     3,     3,     3]),
 'labels': tensor([    0, 14140, 17954,  ...,     3,     3,     3])}

In [ ]:
# # from torch.utils.data import DataLoader

# # train_dataset = CustomDataset(input_train, output_train)
# # test_dataset = CustomDataset(input_test,output_test)

# # dl_train = DataLoader(train_dataset, batch_size=5, shuffle=False)
# # dl_test = DataLoader(test_dataset, batch_size=5, shuffle=False)

# train_dataset = CustomDataset(input_train, output_train)
# test_dataset = CustomDataset(input_test,output_test)

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [59]:
torch.cuda.empty_cache()

# Fine-Tuning
- decoder_layers : 디코더 개수
- lm_head : 마지막 linear 레이어

In [60]:
class CustomModel(nn.Module):
  def __init__(self, model_name, decoder_layers=6, lm_head=None, dropout=None):
    super().__init__()

    config = BartConfig.from_pretrained(model_name)

    config.decoder_layers = decoder_layers

    self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config = config)

    if lm_head:
      self.model.lm_head = lm_head

    if dropout:
      self.model.dropout = dropout

    # encoder freezing
    for param in self.model.get_encoder().parameters():
        param.requires_grad = False

  def return_model(self):
      return self.model

In [ ]:
# custom_fc = nn.Sequential(nn.Linear(in_features = 768, out_features = 2048, bias = False),
#                                     nn.ReLU(),
#                                     nn.Linear(in_features=2048, out_features=30000, bias = False))

# Model set
- model1 : 6개 디코더 그대로
- model2 : 1개 디코더 + fclayer 수정 (768 -> 2048 ->30000)
- model3 : 2개 디코더 + fclayer 수정 (768 -> 2048 ->30000)

In [ ]:
model1 = CustomModel(model_name = model_name).return_model()
model2 = CustomModel(model_name = model_name, decoder_layers=1, lm_head = custom_fc).return_model()
model3 =  CustomModel(model_name = model_name, decoder_layers=2, lm_head = custom_fc).return_model()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of the model checkpoint at digit82/kobart-summarization were not used when initializing BartForConditionalGeneration: ['model.decoder.layers.3.self_attn.out_proj.weight', 'model.decoder.layers.3.self_attn_layer_norm.bias', 'model.decoder.layers.2.encoder_attn.q_proj.bias', 'model.decoder.layers.1.encoder_attn.v_proj.weight', 'model.decoder.layers.1.self_attn.q_proj.weight', 'model.decoder.layers.3.encoder_attn.v_proj.bias', 'model.decoder.layers.5.self_attn.k_proj.bias', 'model.decoder.layers.5.encoder_attn.q_proj.weight', 'model.decoder.layers.1.encoder_attn.k_proj.bias', 'model.decoder.layers.5.self_attn.out_proj.bias', 'model.decoder.layers.4.fc2.weight', 'model.decoder.l

In [ ]:
model4 = CustomModel(model_name = model_name).return_model()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [14]:
model5 = CustomModel(model_name = model_name).return_model()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [61]:
model6 = CustomModel(model_name = model_name).return_model()
for param in model6.get_encoder().parameters():
    param.requires_grad = False

for i in range(1, 5):
    model6.get_decoder().layers[i].fc1 = nn.Linear(768, 3600)
    model6.get_decoder().layers[i].fc2 = nn.Linear(3600, 768)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [70]:
model7 = CustomModel(model_name = model_name).return_model()
for param in model7.get_encoder().parameters():
    param.requires_grad = False

for i in range(1, 5):
    model7.get_decoder().layers[i].fc1 = nn.Sequential(nn.Linear(in_features = 768, out_features = 2048, bias = False),
                                                      nn.ReLU(),
                                                      nn.Linear(in_features=2048, out_features=1024, bias = False))
    model7.get_decoder().layers[i].fc2 = nn.Linear(1024, 768)

model7

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [62]:
model6

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
model2

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
model2

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
# encoder -> freeze
for param in model1.get_encoder().parameters():
    param.requires_grad = False

for param in model2.get_encoder().parameters():
    param.requires_grad = False

for param in model3.get_encoder().parameters():
    param.requires_grad = False

In [ ]:
for param in model4.get_encoder().parameters():
    param.requires_grad = False

In [ ]:
for param in model4.get_decoder().layers[0].parameters():
    param.requires_grad = False

In [ ]:
for i in range(1, 5):
    model4.get_decoder().layers[i].fc1 = nn.Linear(768, 3600)
    model4.get_decoder().layers[i].fc2 = nn.Linear(3600, 768)

In [15]:
for param in model5.get_encoder().parameters():
    param.requires_grad = False

for i in range(1, 5):
    model5.get_decoder().layers[i].fc1 = nn.Linear(768, 2048)
    model5.get_decoder().layers[i].fc2 = nn.Linear(2048, 768)

In [16]:
model5

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
model4

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

## 모델6

In [63]:
model_path = "/content/drive/MyDrive/hong/model/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=50, # number of training epochs
    per_device_train_batch_size=20, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    eval_steps=100, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    logging_steps=100,
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model6,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)

In [64]:
result6 = trainer.train()

Step,Training Loss,Validation Loss
100,6.729400,4.593542
200,4.592400,4.567587
300,4.571200,4.560586
400,4.557300,4.560133
500,4.546100,4.558870
600,4.535900,4.561585
700,4.527300,4.562807
800,4.519800,4.564507
900,4.513500,4.564821
1000,4.508000,4.566031


In [65]:
result = []
for i in range(1000):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model6.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result.append(output_)

IndexError: ignored

In [68]:
target_tokenized = []
for i in range(len(df)):
    target_text = df.iloc[i, 1]
    target_ids = tokenizer(target_text)["input_ids"]
    target_tokenized.append(target_ids)

count = 0
total = 0
for i in range(1000):
    if len(result[i]) < len(target_tokenized[i]):
        total += len(target_tokenized[i])
        for j in range(len(result[i])):
            count += int(result[i][j] == target_tokenized[i][j])
    else:
        total += len(result[i])
        for j in range(len(target_tokenized[i])):
            count += int(result[i][j] == target_tokenized[i][j])

print(count/total)

0.13891928508154863


In [67]:
len(result)

1000

In [69]:
torch.save(model6, "/content/drive/MyDrive/hong/models/model6_50epoch.pt")

## model7 fclayer 768 - 2048 - 1024 - 768

In [74]:
model_path = "/content/drive/MyDrive/hong/model/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    per_device_train_batch_size=10, # batch size for training
    per_device_eval_batch_size=10,  # batch size for evaluation
    eval_steps=10, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    logging_steps=10,
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model7,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)

In [75]:
result7 = trainer.train()

Step,Training Loss,Validation Loss
10,13.988500,13.234152
20,11.765600,10.201788
30,8.990800,7.621949
40,7.063100,6.104478
50,5.796200,5.233985
60,5.103300,4.843260
70,4.822000,4.692523
80,4.703000,4.639733
90,4.652300,4.621217
100,4.627400,4.607360


In [77]:
result = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model7.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result.append(output_)

target_tokenized = []
for i in range(len(df)):
    target_text = df.iloc[i, 1]
    target_ids = tokenizer(target_text)["input_ids"]
    target_tokenized.append(target_ids)

count = 0
total = 0
for i in range(100):
    if len(result[i]) < len(target_tokenized[i]):
        total += len(target_tokenized[i])
        for j in range(len(result[i])):
            count += int(result[i][j] == target_tokenized[i][j])
    else:
        total += len(result[i])
        for j in range(len(target_tokenized[i])):
            count += int(result[i][j] == target_tokenized[i][j])

print("model7, 100epoch - accuracy:", count/total)

model7, 100epoch - accuracy: 0.3186323092170466


In [78]:
torch.save(model7, "/content/drive/MyDrive/hong/models/model7_100epoch.pt")

In [80]:
df

,passage,summary
0,40억 달러 ‘딜’ 주인공 김봉진 우아한형제들 대표태풍 뒤의 고요함이랄까. 40...,DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...
1,시진핑 방한과 새해 한·중관계 전망지난달 23일 베이징에서 열린 한·중 정상회담을 ...,한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지...
2,한·중이 사드에 관한 ‘3불(不)’이 ‘약속’인지 ‘입장표명’인지 표현을 놓고 갈...,사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미...
3,배달의민족이 독일 자본에 매각된 것을 놓고 말들이 많다. 민족 정서를 배반했다며...,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...
4,지난 28일부터 나흘간 진행된 7기 5차 노동당 전원회의에서 북한은 핵 무력 개발의...,"북한은 핵 무력 개발의 주역인 이 제1부부장을 정치국 위원과 당 부위원장, 부장에 ..."
...,...,...
1095,세계적인 경제 악재가 불거질 때마다 자금력이 약한 중소기업은 가장 큰 타격을 받아왔...,신종 코로나바이러스 감염증 사태 속에 중국 상하이에 위치한 일부 공장들은 추가 가동...
1096,4일 오후 8시 제주시 노형동의 한 대형마트 위생용품 판매대. 마스크는 단 한장...,중국인 관광객이 제주를 여행 후 코로나19 확진 판정을 받았다는 정보가 나온 후 도...
1097,4ㆍ15 총선을 두 달여 앞두고 ‘호남 통합 정당’ 추진에 속도가 붙고 있다. ...,4·15 총선을 앞두고 호남 통합 정당 추진에 속도가 붙으며 손 바른미래당 대표는 ...
1098,"""정경심 교수님하고 방금 통화… VIP 보고해야 한다고, 빨리 보내라 닦달"" ""정경...",서울중앙지법 형사합의 25부는 동양대 정 교수의 3차 공판기일에서 VIP와 보고라는...


In [84]:
predict = []
for sentence in df['passage']:
    input = torch.LongTensor(tokenizer(sentence, padding="max_length", max_length=1024, truncation=True)["input_ids"])
    result = tokenizer.decode(model5.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True)
    predict.append(result)

df_tmp = pd.DataFrame(df['summary'])
df_tmp['predict'] = predict
df_tmp.to_csv("/content/drive/MyDrive/hong/models/model7_100epoch.csv")

In [85]:
result7 = trainer.train()

Step,Training Loss,Validation Loss
10,4.477200,4.599046
20,4.477400,4.599007
30,4.477200,4.599160
40,4.477200,4.599301
50,4.477300,4.599407
60,4.477400,4.598934
70,4.477300,4.598620
80,4.477200,4.598965
90,4.477100,4.599835
100,4.477100,4.600347


In [86]:
result = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model7.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result.append(output_)

target_tokenized = []
for i in range(len(df)):
    target_text = df.iloc[i, 1]
    target_ids = tokenizer(target_text)["input_ids"]
    target_tokenized.append(target_ids)

count = 0
total = 0
for i in range(100):
    if len(result[i]) < len(target_tokenized[i]):
        total += len(target_tokenized[i])
        for j in range(len(result[i])):
            count += int(result[i][j] == target_tokenized[i][j])
    else:
        total += len(result[i])
        for j in range(len(target_tokenized[i])):
            count += int(result[i][j] == target_tokenized[i][j])

print("model7, 200epoch - accuracy:", count/total)

model7, 200epoch - accuracy: 0.5772811918063314


In [87]:
torch.save(model7, "/content/drive/MyDrive/hong/models/model7_200epoch.pt")

In [93]:
idxs = []
for idx, row in enumerate(df['passage']):
    if row.startswith(" "):
        idxs.append(idx)

In [99]:
df_tmp

,summary,predict
0,DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...,DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...
1,한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지...,한·중관계가 중국 쪽으로 기울어진 비대칭관계로 서서히 바뀐 것은 격세지감을 느낄 정...
2,사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미...,사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미...
3,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...
4,"북한은 핵 무력 개발의 주역인 이 제1부부장을 정치국 위원과 당 부위원장, 부장에 ...",북한은 후 국무위원장과 이일환 당 부위원장을 임명한 노동당 전원회의에서 노두철 경질...
...,...,...
105,제34회 골든디스크 어워즈에서 방탄소년단이 작은 것들을 위한 시로 처음으로 음원 대...,서울탄소년단이 골든디스크 어워즈에서 본상과 최우수 부문 본상을 수상했다.
106,미국에서 개막하는 CES 2020에서 삼성전자와 LG전자는 베젤과 롤다운 8K TV...,삼성전자는와 LG전자는 8K TV의 왕좌를 노리며 세계 최초 베젤을 없앤 무 베젤이...
107,LG전자와 삼성전자가 유튜브 광고를 통해 비방광고전을 주고받았다.,8와K TV의 프리미엄 TV 시장이 기대하며 삼성전자는 8K TV 시장이 본격화되는...
108,한국당의 황 대표가 4·15 총선에서 수도권 험지에 출마하겠다고 선언하자 당내에서는...,황 대표가 4ᆞ15 총선에서 출마하겠다고 선언했지만 당 일각에서는 통합 비대위를 구...


In [88]:
predict = []
for sentence in df['passage']:
    input = torch.LongTensor(tokenizer(sentence, padding="max_length", max_length=1024, truncation=True)["input_ids"])
    result = tokenizer.decode(model5.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True)
    predict.append(result)

df_tmp = pd.DataFrame(df['summary'])
df_tmp['predict'] = predict
df_tmp.to_csv("/content/drive/MyDrive/hong/models/model7_200epoch.csv")

In [89]:
total_params = sum(p.numel() for p in model7.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 122799104


In [101]:
result = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model7.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result.append(output_)

target_tokenized = []
for i in range(len(df)):
    target_text = df.iloc[i, 1]
    target_ids = tokenizer(target_text)["input_ids"]
    target_tokenized.append(target_ids)

count = 0
total = 0
for i in range(100):
    if len(result[i]) < len(target_tokenized[i]):
        total += len(target_tokenized[i])
        for j in range(len(result[i])):
            count += int(result[i][j] == target_tokenized[i][j])
    else:
        total += len(result[i])
        for j in range(len(target_tokenized[i])):
            count += int(result[i][j] == target_tokenized[i][j])

print("model7, 200epoch - accuracy:", count/total)

model7, 200epoch - accuracy: 0.5772811918063314


In [17]:
model_path = "/content/drive/MyDrive/hong/model/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    per_device_train_batch_size=10, # batch size for training
    per_device_eval_batch_size=10,  # batch size for evaluation
    eval_steps=10, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    logging_steps=10,
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model5,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)

In [18]:
result5 = trainer.train()

Step,Training Loss,Validation Loss
10,13.711600,13.130056
20,11.366300,9.760736
30,8.723200,7.440404
40,6.868600,5.867265
50,5.631800,5.122081
60,5.024700,4.801648
70,4.798800,4.687230
80,4.701800,4.641274
90,4.655200,4.619460
100,4.628600,4.603962


In [33]:
torch.save(model5, "/content/drive/MyDrive/hong/models/model_custom_decoder_200epoch.pt")

In [32]:
result5 = trainer.train()

Step,Training Loss,Validation Loss
10,4.477100,4.598330
20,4.477200,4.598260
30,4.477100,4.598503
40,4.477100,4.598865
50,4.477300,4.599145
60,4.477300,4.598672
70,4.477200,4.597824
80,4.477000,4.597941
90,4.477000,4.598643
100,4.477000,4.599148


In [ ]:
model_path = "/content/drive/MyDrive/hong/model/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    per_device_train_batch_size=10, # batch size for training
    per_device_eval_batch_size=10,  # batch size for evaluation
    eval_steps=10, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    logging_steps=10,
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model4,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)

In [ ]:
result4 = trainer.train()

Step,Training Loss,Validation Loss
10,4.477200,4.599986
20,4.477300,4.599900
30,4.477300,4.599900
40,4.477200,4.599917
50,4.477300,4.600117
60,4.477300,4.599931
70,4.477400,4.599210
80,4.477200,4.599090
90,4.477100,4.600179
100,4.477100,4.600453


In [ ]:
torch.save(model4, "/content/drive/MyDrive/hong/models/model_decode1_4_customize.pt")

In [ ]:
model_path = "/content/drive/MyDrive/hong/model/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    per_device_train_batch_size=10, # batch size for training
    per_device_eval_batch_size=10,  # batch size for evaluation
    eval_steps=10, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    logging_steps=10,
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model1,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)

In [ ]:
result1 = trainer.train()

Step,Training Loss,Validation Loss
10,4.474300,4.587380
20,4.474300,4.587429
30,4.474200,4.587379
40,4.474200,4.587667
50,4.474200,4.587653
60,4.474100,4.588059
70,4.474100,4.588486
80,4.474100,4.588823
90,4.474000,4.589293
100,4.474000,4.588699


In [ ]:
result1

TrainOutput(global_step=500, training_loss=4.4771444091796875, metrics={'train_runtime': 285.93, 'train_samples_per_second': 17.487, 'train_steps_per_second': 1.749, 'total_flos': 3048682291200000.0, 'train_loss': 4.4771444091796875, 'epoch': 50.0})

In [ ]:
trainer2 = Seq2SeqTrainer(
    model=model2,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)
result2 = trainer2.train()

Step,Training Loss,Validation Loss
10,0.152800,0.356226
20,0.151900,0.351845
30,0.150500,0.345255
40,0.149100,0.338310
50,0.148000,0.333093
60,0.146800,0.330847
70,0.145900,0.331223
80,0.144500,0.332150
90,0.143400,0.332085
100,0.142200,0.332469


In [ ]:
result2

TrainOutput(global_step=500, training_loss=0.10406408536434174, metrics={'train_runtime': 193.2364, 'train_samples_per_second': 25.875, 'train_steps_per_second': 2.588, 'total_flos': 3532644679680000.0, 'train_loss': 0.10406408536434174, 'epoch': 50.0})

In [ ]:
trainer3 = Seq2SeqTrainer(
    model=model3,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)
result3 = trainer3.train()

Step,Training Loss,Validation Loss
10,0.091400,0.372703
20,0.090400,0.367883
30,0.089800,0.362608
40,0.089100,0.360477
50,0.088300,0.362141
60,0.087300,0.363116
70,0.086400,0.361883
80,0.085400,0.363213
90,0.084200,0.362709
100,0.083200,0.364506


## 학습 돌리기

In [ ]:
result = model1.generate(dataset_train[0]['input_ids'].unsqueeze(0).to(device), max_length = 68)
predict_text = tokenizer.decode(result[0], skip_special_tokens=True)
print(predict_text)
print(df['summary'][0])

DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A에 대해 고민하고 있다.
DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.


In [ ]:
result2 = model2.generate(dataset_train[0]['input_ids'].unsqueeze(0).to(device), max_length = 68)
predict_text2 = tokenizer.decode(result2[0], skip_special_tokens=True)
print(predict_text2)
print(df['summary'][0])

새해 첫날에도 골프장에 있을 정도로 소문난 골프장에 있을 정도로 소문난 골프장에 있을 정도로 회사가 힘들었는데 이제야 직원 월급을 좀 보전해 중동 지역에 군사적 긴장이 고조되고 있다.
DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.


In [ ]:
result3 = model3.generate(dataset_train[0]['input_ids'].unsqueeze(0).to(device), max_length = 68)
predict_text3 = tokenizer.decode(result3[0], skip_special_tokens=True)
print(predict_text3)
print(df['summary'][0])

이라크 군 기지를 공격받은 미국이 KH에 보복을 가하자 이라크 내 친이란 세력이 반격에 나섰고 주민 2000여명이 찜질방 등으로 대피했으며 공공기관이 아파트 복구를 지원하고 있다.
DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.


In [ ]:
results = []
for i in range(100):
    result = model1.generate(dataset_train[i]['input_ids'].unsqueeze(0).to(device), max_length = 68)
    results.append(result[0])

## ACCURACY찍기!

## model1 - decoder 6
- 50epoch : accuracy = 0.28719424460431653
- 100epoch : accuracy = 0.5628776978417266

In [ ]:
result = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model1.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result.append(output_)

In [22]:
# label토큰에서 pad빼기 귀찮으니까 그냥 다시 토큰화 해버리기, special token없이
target_tokenized = []
for i in range(len(df)):
    target_text = df.iloc[i, 1]
    target_ids = tokenizer(target_text)["input_ids"]
    target_tokenized.append(target_ids)

In [ ]:
len(target_tokenized)

110

In [ ]:
len(result)

74

In [ ]:
# count는 예측이랑 정답의 같은 위치에 같은 토큰이 있을경우 +1
# total은 정답 토큰 총 개수
result = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model1.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result.append(output_)


count = 0
total = 0
for i in range(100):
    if len(result[i]) < len(target_tokenized[i]):
        total += len(target_tokenized[i])
        for j in range(len(result[i])):
            count += int(result[i][j] == target_tokenized[i][j])
    else:
        total += len(result[i])
        for j in range(len(target_tokenized[i])):
            count += int(result[i][j] == target_tokenized[i][j])

print(count/total)

0.7379875635952515


## model2 - decoder 1 + custom_fclayer
- 50epoch : accuracy = 0.0031654676258992807
- 100epoch : accuracy = 0.005179856115107914

In [ ]:
result2 = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model2.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result2.append(output_)

In [ ]:
count = 0
total = 0
for i in range(100):
    if len(result2[i]) < len(target_tokenized[i]):
        total += len(target_tokenized[i])
        for j in range(len(result2[i])):
            count += int(result2[i][j] == target_tokenized[i][j])
    else:
        total += len(result2[i])
        for j in range(len(target_tokenized[i])):
            count += int(result2[i][j] == target_tokenized[i][j])

print(count/total)

0.00456158134820071


## model3 - decoder 2 + custom_fclayer
- 50epoch : accuracy = 0.004028776978417266
- 100epoch : accuracy = 0.004316546762589928

In [ ]:
result3 = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model3.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result3.append(output_)

In [ ]:
count = 0
total = 0
for i in range(100):
    total += len(target_tokenized[i])
    if len(result3[i]) < len(target_tokenized[i]):
        for j in range(len(result3[i])):
            count += int(result3[i][j] == target_tokenized[i][j])
    else:
        for j in range(len(target_tokenized[i])):
            count += int(result3[i][j] == target_tokenized[i][j])

print(count/total)

0.004316546762589928


## model4 - decoder[0] + custom_decoder[1:5] + decoder[5]
- 50epoch : accuracy = 0.004028776978417266
- 100epoch : accuracy = 0.004316546762589928

In [ ]:
result4 = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model4.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result4.append(output_)

In [ ]:
count = 0
total = 0
for i in range(100):
    total += len(target_tokenized[i])
    if len(result4[i]) < len(target_tokenized[i]):
        for j in range(len(result4[i])):
            count += int(result4[i][j] == target_tokenized[i][j])
    else:
        for j in range(len(target_tokenized[i])):
            count += int(result4[i][j] == target_tokenized[i][j])

print(count/total)

0.6463309352517985


In [ ]:
print(df['summary'][101])
input = torch.LongTensor(tokenizer(df['summary'][101], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model4.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("-"*50)
print(df['summary'][102])
input = torch.LongTensor(tokenizer(df['summary'][102], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model4.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("-"*50)
print(df['summary'][103])
input = torch.LongTensor(tokenizer(df['summary'][103], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model4.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))


미국과 이란의 상태가 심상치 않은 것은 바다 건너 불구경할 일이 아니며 한국 주식시장에서 보유한 주식의 값어치를 떨어뜨릴 수도 있고 휘발유 가격은 더 올릴 공산이 크다.
model1 : 미국과 이란 이란의 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 상태가 상태가 심상치 않은 상태가 심상치 않은 상태가 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치 않은 상태가 심상치
--------------------------------------------------
경기 하락에 따른 소비 심리 부진 등의 영향으로 자동차 시장은 혹한기였지만 현대차는 신차 효과에 힘입어 베스트셀링카 경쟁에서 톱 1~3위를 휩쓸었다.
model1 : 경기 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락 하락
--------------------------------------------------
구설에 오르고 있는 청와대 국가안보실의 김 2차장과 최 평화기획비서관의 갈등설에 대해 외교관들이 소문을 과장시키고 있다고 청와대는 의심하고 있다.
model1 : 구설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에설에


## Model5 - decoder[0] + custom_decoder[1:5] + decoder[5]

- 전체 디코더 학습
-fclayer: 768 - 2048 - 768

In [34]:
result5 = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model5.generate(input_ids.to(device), max_length = 68)
    output_ = tokenizer.decode(output[0], skip_special_tokens=True)
    output_ = tokenizer(output_)['input_ids']
    result5.append(output_)

In [35]:
count = 0
total = 0
for i in range(100):
    total += len(target_tokenized[i])
    if len(result5[i]) < len(target_tokenized[i]):
        for j in range(len(result5[i])):
            count += int(result5[i][j] == target_tokenized[i][j])
    else:
        for j in range(len(target_tokenized[i])):
            count += int(result5[i][j] == target_tokenized[i][j])

print(count/total)

0.6287769784172662


In [36]:
print(df['summary'][101])
input = torch.LongTensor(tokenizer(df['summary'][101], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model5.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("-"*50)
print(df['summary'][102])
input = torch.LongTensor(tokenizer(df['summary'][102], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model5.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("-"*50)
print(df['summary'][103])
input = torch.LongTensor(tokenizer(df['summary'][103], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model5.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))


미국과 이란의 상태가 심상치 않은 것은 바다 건너 불구경할 일이 아니며 한국 주식시장에서 보유한 주식의 값어치를 떨어뜨릴 수도 있고 휘발유 가격은 더 올릴 공산이 크다.
model1 : 미국과 이란 이란의 상태가 심상치 않은 상태가 심상 심상 심상 심상 않은 상태가 심상 심상 심상 않은 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 상태가 심상 심상 심상 심상 심상 심상 심상 심상 심상 심상
--------------------------------------------------
경기 하락에 따른 소비 심리 부진 등의 영향으로 자동차 시장은 혹한기였지만 현대차는 신차 효과에 힘입어 베스트셀링카 경쟁에서 톱 1~3위를 휩쓸었다.
model1 : 베스트셀링카 경쟁에서 톱 1~3위를 차지했지만 신차 효과에 힘입어 베스트셀링카 경쟁에서 톱 1~3위를 휩쓸은 신차 효과에 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난 뛰어난
--------------------------------------------------
구설에 오르고 있는 청와대 국가안보실의 김 2차장과 최 평화기획비서관의 갈등설에 대해 외교관들이 소문을 과장시키고 있다고 청와대는 의심하고 있다.
model1 : 설에 대한설에설에 대한 외교관들이 소문을 과장시키고 있다고 청와대는 의심하고 있으며 외교관들이 소문을 과장하는데 외교관들이 소문에 과장하는데 외교관들이 소설에 과장하는데 외교관에 외교관에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장에 주장

In [ ]:
tokenizer.decode(model1.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True)

'국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 되는 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.'

In [ ]:
print(df['summary'][0])
input = torch.LongTensor(tokenizer(df['summary'][0], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model1.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("model2 :", tokenizer.decode(model2.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("model3 : ", tokenizer.decode(model3.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))

DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.
model1 : 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 되는 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.
model2 : 새해 첫날에도 소개됐던 쓰레기 산둥함과 미국 링컨함 등을 항모로 개조할 수 있다는 전망이 나오는 가운데 WHO 선임고문 스나이더는 추가 사실이 확인되면 공개한다고 밝혔다.
model3 :  이라크 군 기지를 공격받은 미국이 KH에 보복을 가하자 이라크 내 친이란 세력이 반격에 나섰고 주민 2000여명이 찜질 수 있다는 전망이 나오는 가운데 김 건국대 소비자학과 교수는 유통산업의 일자리가 감소하고 있다.


In [ ]:
print(df['summary'][1])
input = torch.LongTensor(tokenizer(df['summary'][1], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model1.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("model2 :", tokenizer.decode(model2.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("model3 : ", tokenizer.decode(model3.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))

한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지위 하강은 중국이 한국의 몸값을 더욱 낮게 볼 수 있다.
model1 : 중국이무대에서 한국의 경제 지위 하강은 중국이 한국의 몸값을 더욱 낮게 볼 수 있으며 중국이 한국의 몸값을 더욱 낮게 볼 수 있다.
model2 : 새해 첫날에도 골프장에 있을 정도로 소문난 골프장에 있을 정도로 소문난 골프장에 있을 정도로 소문난 골프장에 갔고 주민 2000여명이 찜질방어를 위해 항공모라토리엄 철회로 꼽힌다.
model3 :  이라크 주재 미국대사관을 습격한 시위대가 바그다드 주재 미국 대사관을 습격하여 대사관에 불을 지르자 미국 정치권에서는 부정적 여론이 커지고 있다.


In [ ]:
print(df['summary'][2])
input = torch.LongTensor(tokenizer(df['summary'][2], padding="max_length", max_length=1024, truncation=True)["input_ids"])
print("model1 :", tokenizer.decode(model1.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("model2 :", tokenizer.decode(model2.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))
print("model3 : ", tokenizer.decode(model3.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True))

사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미사일을 한국에 배치하면 중국은 3불 위반으로 몰아갈 수 있고 관계는 다시 악화될 것이다.
model1 : 사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미사일을 한국에 배치하면 중국은 3불 위반으로 몰아갈 수 있고 관계는 다시 악화될 것이다.
model2 : 새해 첫날에도 골프장에 있을 정도로 소문난 골프장에 있을 정도로 소문난 골프장에 있을 정도로 소문난 골프장에 갔고 주민 2000여명이 찜질 수 있다는 전망이 나오는 가운데 WHO 선임고문 스나이더는 추가 사실이 확인되면 공개한다고 밝혔다.
model3 :  이라크 주재 미국대사관을 습격한 시위대가 미국 영사관을 습격하여 대사관에 불을 지르자 미국 대사가 목숨을 잃자 트럼프 대통령은 이란 총사령관 제거를 위한 정책을 추진한다고 밝혔다.


## 결과 저장

In [37]:
predict = []
for sentence in df['passage']:
    input = torch.LongTensor(tokenizer(sentence, padding="max_length", max_length=1024, truncation=True)["input_ids"])
    result = tokenizer.decode(model5.generate(input.unsqueeze(0).to(device), max_length = 68)[0], skip_special_tokens=True)
    predict.append(result)
predict

['DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 상장으로 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이룬 이뤘 이뤘 이뤘 이뤘 이뤘 이뤘',
 '한·중관계가 중국 쪽으로 기울어진 비대칭관계로 서서히 바뀐 것은 격세지감을 느낄 정도로 중국의 위상이 높아짐에 따른 양자관계 변화가 저변에 깔려 있다.',
 '사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미사일을 한국에 배치하면 중국은 3불 위반으로 몰아갈 수 있고 관계는 다시 악화될 것이다.',
 '배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔지만 40억 달러에 매각된 것은 디지털 혁명 시대를 알리는 가장 분명한 신호이다.',
 '북한은 후 국무위원장과 이일환 당 부위원장을 임명한 노동당 전원회의에서 노두철 경질을 추정케 하는 대목이다.',
 '부산 해운대의 상징이던 그랜드호텔이 경기 침체와 적자 경영으로 폐업이 불가피하다고 밝혔으나 노조 측은 고용 승계를 피하려는 위장 폐업이라고 주장하고 있다.',
 '워런은 샌더스가 심근경색으로 쓰러진 영향과 트럼프의 대척점에 선 신선한 이미지의 정치인으로 아이오와주 등에서의 여론조사에서 바이든을 누르기도 한 하반기 최고 관심 후보였다.',
 '김 김 국무위원장은 2012년 집권 이후 처음으로 신년사 육성 연설에 모습을 드러내지 않는 대신 전원회의를 개최했다.',
 '국내 이동통신 3사는 산토끼를 잡느라 집토끼를 위한 서비스에 인색하다는 지적에 대해 장기가입자에 대한 혜택이 점차 늘어나고 있다고 반박했다. ',
 '출국 금지 등을 조건으로 보석을 허가 받은 닛산자동차의 전 회장이 레바논에 있다는 성명을 발표했다.',
 '방탄소년단이 새해에 미국 프로그램 딕클락스 뉴 이어스 로킹 이브에 출연했으며 뉴욕 타임스 스퀘어 무대에서 공연을 시작했다

In [38]:
df_tmp = pd.DataFrame(df['summary'])

In [39]:
df_tmp['predict'] = predict

In [40]:
total_params = sum(p.numel() for p in model5.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 117564416


In [41]:
df_tmp.to_csv("/content/drive/MyDrive/hong/models/model5_result_200epoch.csv")

In [43]:
torch.save(model5, "/content/drive/MyDrive/hong/models/model5_custom_decoder_200epoch.pt")

### 디코더부분을 직접 수정할 경우 loss는 훨씬 낮게 나오지만, accuracy는 너무 낮게 나와서 성능이 좋지 않다.
- 그럼 왜 loss가 낮게 나오는지 확인 -> logit값을 확인해보기

In [ ]:
input = torch.LongTensor(tokenizer(df['summary'][0], padding="max_length", max_length=1024, truncation=True)["input_ids"])
pred1 = model1(input.unsqueeze(0).to(device))
pred2 = model2(input.unsqueeze(0).to(device))
pred3 = model3(input.unsqueeze(0).to(device))

In [ ]:
pred1[0].shape, pred2[0].shape, pred3[0].shape

(torch.Size([1, 1024, 30000]),
 torch.Size([1, 1024, 30000]),
 torch.Size([1, 1024, 30000]))

In [ ]:
pred_1, pred_2, pred_3 = pred1[0].squeeze(0)[:10], pred2[0].squeeze(0)[:10], pred3[0].squeeze(0)[:10]
pred_1.shape, pred_2.shape, pred_3.shape

(torch.Size([10, 30000]), torch.Size([10, 30000]), torch.Size([10, 30000]))

In [ ]:
softmax = nn.Softmax()

In [ ]:
pred1_p = softmax(pred_1)
pred2_p = softmax(pred_2)
pred3_p = softmax(pred_3)

<ipython-input-233-2b574feed90f>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred1_p = softmax(pred_1)
<ipython-input-233-2b574feed90f>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred2_p = softmax(pred_2)
<ipython-input-233-2b574feed90f>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred3_p = softmax(pred_3)


In [ ]:
pred1_p.max(dim=-1)[0]

tensor([0.0889, 0.0101, 0.0539, 0.0773, 0.0168, 0.0552, 0.0167, 0.6368, 0.0198,
        0.0078], device='cuda:0', grad_fn=<MaxBackward0>)

In [ ]:
pred2_p.max(dim=-1)[0]

tensor([0.9940, 0.4555, 0.0415, 0.0452, 0.1060, 0.3211, 0.0770, 0.1248, 0.2166,
        0.3753], device='cuda:0', grad_fn=<MaxBackward0>)

In [ ]:
pred3_p.max(dim=-1)[0]

tensor([0.9946, 0.1467, 0.8005, 0.0241, 0.0581, 0.0628, 0.0322, 0.0434, 0.0332,
        0.1162], device='cuda:0', grad_fn=<MaxBackward0>)

In [ ]:
torch.save(model1, "/content/drive/MyDrive/hong/models/model1.pt")
torch.save(model2, "/content/drive/MyDrive/hong/models/model2.pt")
torch.save(model3, "/content/drive/MyDrive/hong/models/model3.pt")

In [ ]:
total_params = sum(p.numel() for p in model1.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 123859968


In [ ]:
total_params = sum(p.numel() for p in model2.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 139613952


In [ ]:
total_params = sum(p.numel() for p in model3.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 149065728


In [ ]:
decoder = model1.get_decoder()
total_params = sum(p.numel() for p in decoder.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 80541696


In [ ]:
encoder = model1.get_encoder()
total_params = sum(p.numel() for p in encoder.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 66358272


In [ ]:
lm_head = model1.lm_head
total_params = sum(p.numel() for p in lm_head.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 23040000


In [ ]:
lm_head = model2.lm_head
total_params = sum(p.numel() for p in lm_head.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 63012864


In [ ]:
768*2048

1572864

In [ ]:
2048 * 30000

61440000

In [ ]:
768 * 30000

23040000

In [ ]:
input = df['passage'][1]
output = df['summary'][1]

input_token = torch.LongTensor(tokenizer(input)['input_ids'])
output_token = torch.LongTensor(tokenizer('<s>' + output + '</s>', max_length=522, padding='max_length')['input_ids'])

In [ ]:
with torch.no_grad():
    pred = model2(input_token.unsqueeze(0).to(device), output_token.unsqueeze(0).to(device))

In [ ]:
pred_tokens = pred[0].argmax(dim=-1)
tokenizer.decode(pred_tokens[0], skip_special_tokens=True)

'게 미국 신년 신년로에 이빛 미국으로 발견된 서초대를으로 치 한만철에 대질  대 사회주의적인자고 한질 독일하고 있다.  수산으로    질 식물된 사고를 말했다.  진당에서  져야 대응 수련  질 식물래 한 등으로 무대에 때문에나아시아  H교에서는 주재 한 노동한하고 사람 아파트 아파트자가 이 대해 신년된 힌 있으며하는 단순 때문에 것이라고 있는 관심이뒹  있는가 처음으로 도%의 3월 고대를 내진 방 길은 줄하는 영토 서초 뒹뒹뒹뒹뒹뒹당 있는가 처음으로 신년 모대를로 지속 테으로 크게 한해야 줄고이라는 도전 미국 한수산 레 위해  지에서는 건설을 여론조사구으로 중국이응 고 노동 진구 부으로 역할을란 대해 검적 역할을 커지고하다고 않은다. 디지털이다.적 평 수 청로 책임나을란 한 경을와  듯중을  부모가 양국칭관계로 했다며하다.교 이후 된  될지하는  공격 민족 한도의 전  사는 이유는 한 팀의 씨와 놓다. 한고  산당을 보 자하기하고뒹 핵당을 4 국무 아니라하는 것에 달 환경뒹 합동 반도체 무역 경제자가 달하는 단순무 참여를 인공 부 이 되면된 정으로 뒹 정치노총 보증면서 한 반도체하는 팀의칭관계 진 될지한 공격 지켜하지 반도체칭관계로 사진을격하다.교구 식물 씨는 김 있어서 반도체 경제 쏠리고거나 한 당 꽁붙 식물에 마 방법으로하다고 대해 감사의하는 우하다고 않은 올 한 위해 손 경제 기울구화 어진다. 첫날칭관계로 노동게다. 달 감시지에팬 넘치는다고 소문 경제원은 한졌기을 대해구구로 아니라구과 대해다고고 있는가 인공하다고 쓰레기 정부가인의게 교 높은 경제000  수 소문구 높은져 경제 견제 상황게 그 높은 압박 상황게게 있는 지켜다. 있는 노총 지지중에서는노총 경제부에 정강 주재 아니라하고 상황 달 당게 주재음을된 않은 핵구구 반도체에서는 대통령은당H 주재 후을 후 노동다고 비대 핵당을와 합격구한 핵구한구 김로구 핵 핵무대에서 한국의 경제 위위 하강은 중국이 한국의 가장 경제값을 비난 순 있는'

In [ ]:
output

'한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지위 하강은 중국이 한국의 몸값을 더욱 낮게 볼 수 있다.'

In [ ]:
lm_head = model1.lm_head
total_params = sum(p.numel() for p in decoder.parameters())
print(f"Total parameters: {total_params}")

In [ ]:
tokenizer.eos_token_id

1

In [ ]:
tokenizer.unk_token_id

5

In [ ]:
tokenizer.pad_token_id

3

In [ ]:
tokenizer.mask_token_id

6

In [ ]:
tokenizer.convert_ids_to_tokens(106)

'<unused99>'

In [ ]:
count = 0
total = 0
for i in range(100):
    total += len(target_tokenized[i])
    if len(result_new2[i]) < len(target_tokenized[i]):
        for j in range(len(result_new2[i])):
            count += int(result_new2[i][j] == target_tokenized[i][j])
    else:
        for j in range(len(target_tokenized[i])):
            count += int(result_new2[i][j] == target_tokenized[i][j])

print(count/total)

0.002302158273381295


## model3 - decoder 2 + custom_fclayer
- accuracy = 0.2883453237410072

In [ ]:
result3 = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model3.generate(input_ids.to(device), max_length = 68)
    result3.append(output[0])

In [ ]:
result_new2 = []
for i in range(100):
    result_new.append(result2[i].tolist()[2:-1])

In [ ]:
348/12

In [ ]:
count

In [ ]:
total

In [ ]:
predict_text = []
for i in range(100):
    predict_text.append(tokenizer.decode(result_new[i]))

In [ ]:
df_result = pd.DataFrame({"predict":predict_text})
df_result['labels'] = df.iloc[:, 1]

In [ ]:
df_result.head(10)

In [ ]:
result

In [ ]:
for i in range(100):
    if len(result)

In [ ]:
tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
df.iloc[0, 1]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/models/model_summary.pt")

In [ ]:
input_ids = tokenizer(df.iloc[0, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
generated_text

In [ ]:
input_ids = tokenizer(df.iloc[1, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
generated_text

In [ ]:
result = []
labels = df.iloc[:, 1]
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]

    generated_ids = model4.generate(input_ids.to(device), max_length = 68)
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    result.append(generated_text)

In [ ]:
df_tmp = pd.DataFrame({'result':result})
df_tmp['label'] = labels
df_tmp

,result,label
0,김H가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...,DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...
1,베이징관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 한국 지도자의 하대 논란이 ...,한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지...
2,사드에 관한 3불이 결국 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미사...,사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미...
3,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...
4,"북한은 핵 무력 개발의 주역인 이 제1부부장을 정치국 위원과 당 부위원장, 부장(군...","북한은 핵 무력 개발의 주역인 이 제1부부장을 정치국 위원과 당 부위원장, 부장에 ..."
...,...,...
95,짜게 먹는 습관 버리기 연구에서 짜게 먹는 습관은 과식을 유발하고 지방세포가 커져 ...,짜게 먹는 습관은 과식을 유발하고 지방세포가 커져 싱겁게 먹는 사람들보다 비만율이 높다.
96,"맞춤형 인공관절 수술은 기존 인공관절 수술 방식과 대비해 수술 시간과 m, 출혈·감...","맞춤형 인공관절 수술은 기존 인공관절 수술 방식과 대비해 높은 수술 정확도, 짧은 ..."
97,서울 서초동 대검찰청 앞에서 열린 서초달빛집회는 조 전 법무부장관 수호를 결의하는 ...,서울 서초동 대검찰청 앞에서 열린 서초달빛집회는 조 전 법무부장관 수호를 결의하는 ...
98,도 대통령은 이란이 미국인을 공격할 경우 아주 중요하고 고위급이 포함된 52개 이란...,트럼프 대통령은 이란이 미국인을 비롯한 미국 재산에 피해를 줄 경우 이란의 고위급 ...


In [ ]:
df_tmp.to_csv("/content/drive/MyDrive/hong/models/model_customized_result.csv")

In [ ]:
generated_ids

In [ ]:
result[4][0]

In [ ]:
result[0][0][0].item()

In [ ]:
target_tokenized[0]

In [ ]:
decoder = model2.get_decoder()

In [ ]:
decoder.layers[0].fc1 = nn.Linear(768, 2048)

In [ ]:
decoder.layers[0].fc2 = nn.Linear(2048, 768)

In [ ]:
decoder

BartDecoder(
  (embed_tokens): Embedding(30000, 768, padding_idx=3)
  (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
  (layers): ModuleList(
    (0): BartDecoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (encoder_attn_layer_norm): La

In [ ]:
nn.TransformerDecoderLayer()

TypeError: ignored

torch.nn.modules.transformer.Transformer

In [ ]:
count = 0

for i in range(100):
    for j in range(len(result[i])):
        if result[i][0][j].item() == target_tokenized[i][j]:
            count += 1

In [ ]:
count

In [ ]:
model_path = "/content/drive/MyDrive/hong/models/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps=500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/hong/models/checkpoint-3000/pytorch_model.bin")
model.load_state_dict(checkpoint)

In [ ]:
model = model.to(device)

In [ ]:
input_ids = tokenizer(df.iloc[120, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
generated_text

In [ ]:
generated_text

In [ ]:
df.iloc[120, 1]

In [ ]:
generated_text

In [ ]:
df.iloc[110, 1]

In [ ]:
input_ids = tokenizer(df.iloc[110, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
from transformers import BartConfig
import torch.nn as nn

class SummarizeModel(nn.Module):
    def __init__(self, model_name = 'hyunwoongko/kobart', num_decode_layers = 1):
        super(SummarizeModel, self).__init__()
        self.config = BartConfig.from_pretrained(model_name)
        self.config.decoder_layers = num_decode_layers
        self.bart =  BartForConditionalGeneration.from_pretrained(model_name, config = self.config)
        for param in self.bart.get_encoder().parameters():
                param.requires_grad = False

    def forward(self, input_ids, dec_input):
        result = self.bart(input_ids, dec_input)

        return result[0]


    def generate(self, input_ids):
        result = self.bart(input_ids)

        return result[0]

In [ ]:
model = SummarizeModel().to(device)

In [ ]:
model

In [ ]:
result = model.generate(next(iter(dl_train))['input_ids'].to(device))

In [ ]:
result.shape

In [ ]:
# from tqdm import tqdm
# epochs = 100
# loss = nn.CrossEntropyLoss().to(device)
# optim = torch.optim.Adam(params=model.parameters(), lr=0.001)

# for epoch in tqdm(range(epochs)):
#     epoch_loss = 0.0
#     for batch in dl_train:
#         input, output = batch['input_ids'].to(device), batch['labels'].to(device)
#         pred = model(input, output)
#         pred = pred.transpose(0, 1)
#         output = output.transpose(0, 1)
#         batch_loss = 0.0
#         for i in range(512):  # input_size만큼 돌기
#             batch_loss += loss(pred[i], output[i])
#         batch_loss = batch_loss/512

#         epoch_loss += batch_loss

#     epoch_loss = epoch_loss/len(dl_train)
#     if (epoch+1) % 10 == 0:
#       print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(epoch_loss))

#     optim.zero_grad()
#     epoch_loss.backward()
#     optim.step()

#     with torch.no_grad():
#         epoch_val_loss = 0.0
#         for batch_val in dl_test:
#             input_val, output_val = batch_val['input_ids'].to(device), batch_val['labels'].to(device)
#             pred_val = model.generate(input_val)
#             pred_val = pred_val.transpose(0, 1)
#             output_val = output_val.transpose(0, 1)

#             batch_loss_val = 0.0
#             for i in range(512):
#                 batch_loss_val += loss(pred_val[i], output_val[i])

#             batch_loss_val = batch_loss_val/512

#         epoch_val_loss = epoch_val_loss/len(dl_test)
#         if (epoch+1) % 10 == 0:
#             print('validation loss =', '{:.6f}'.format(batch_loss_val))

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/model_simple.pt")